# Clockodo API Testing

This notebook tests the Clockodo client functions to debug API interactions.

## Setup

Make sure you have a .env file in the project root with your Clockodo credentials.

## Usage

You can now run all cells sequentially! The notebook automatically:
- Fetches active (non-archived) services dynamically in each test
- Uses ISO 8601 datetime format for time entries
- Handles API requirements correctly

In [ ]:
import os
import sys
from pathlib import Path

# Add the src directory to Python path
notebook_dir = Path.cwd()
src_dir = notebook_dir.parent / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

print(f"Added {src_dir} to Python path")
print(f"Current working directory: {notebook_dir}")

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load .env from project root (parent directory of manual-test)
env_path = notebook_dir.parent / '.env'
if env_path.exists():
    load_dotenv(env_path, override=True)
    print(f"✓ Loaded environment variables from {env_path}")
else:
    print(f"⚠️  Warning: .env file not found at {env_path}")
    print("Please create a .env file with CLOCKODO_API_USER and CLOCKODO_API_KEY")

# Global variables for test cleanup and coordination
clock_entry_id = None
last_absence_id = None
last_entry_id = None
team_vacation_id = None
team_vacation_id_reject = None
first_customer_id = None

# Current year for reports
from datetime import datetime
current_year = datetime.now().year

# Optional: Overwrite team member ID for Team Leader tests
# Set TEST_TEAM_MEMBER_ID in your .env file
env_team_member_id = os.getenv("TEST_TEAM_MEMBER_ID")
if env_team_member_id:
    team_member_id = int(env_team_member_id)
    print(f"✓ Found TEST_TEAM_MEMBER_ID in env: {team_member_id}")
else:
    team_member_id = None
    print("ℹ️  TEST_TEAM_MEMBER_ID not found in env, will use dynamic discovery")


## Import Clockodo Client

In [ ]:
from clockodo_mcp.client import ClockodoClient

# Create client from environment variables
client = ClockodoClient.from_env()

# Create services
from clockodo_mcp.services.user_service import UserService
from clockodo_mcp.services.hr_service import HRService
from clockodo_mcp.services.team_leader_service import TeamLeaderService

user_service = UserService(client)
hr_service = HRService(client)
team_leader_service = TeamLeaderService(client)

print(f"Client created and services initialized:")
print(f"  API User: {client.api_user}")
print(f"  Base URL: {client.base_url}")
print(f"  User Agent: {client.user_agent}")

## Test 1: List Users

In [ ]:
import json

try:
    users_response = client.list_users()
    print("✓ Successfully retrieved users")
    print(f"\nRaw response:")
    print(json.dumps(users_response, indent=2))
    
    # Parse users if available
    if 'users' in users_response:
        users = users_response['users']
        print(f"\n📊 Found {len(users)} user(s)")
        for user in users:
            print(f"  - ID: {user.get('id')}, Name: {user.get('name')}, Email: {user.get('email')}")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 1a: List Customers

In [ ]:
try:
    customers_response = client.list_customers()
    print("✓ Successfully retrieved customers")
    
    if 'customers' in customers_response and len(customers_response['customers']) > 0:
        customers = customers_response['customers']
        print(f"\n📊 Found {len(customers)} customer(s)")
        # Store the first one for later tests
        first_customer_id = customers[0]['id']
        print(f"  - First Customer: {customers[0].get('name')} (ID: {first_customer_id})")
    else:
        print("No customers found")
        first_customer_id = None
except Exception as e:
    print(f"❌ Error: {e}")

# Helper function to get active service ID
def get_first_active_service_id():
    """Get the first active (non-archived) service ID."""
    try:
        services_response = client.list_services()
        if 'services' in services_response:
            services = services_response['services']
            # Filter for active services only
            active_services = [s for s in services if s.get('active') is True]
            if active_services:
                return active_services[0]['id']
        return None
    except Exception:
        return None


## Test 1b: List Services

This cell displays all services (active and archived). Later test cells automatically use the first active service.

In [ ]:
try:
    services_response = client.list_services()
    print("✓ Successfully retrieved services")
    
    if 'services' in services_response and len(services_response['services']) > 0:
        services = services_response['services']
        print(f"\n📊 Found {len(services)} service(s)")
        
        # Filter for active services (active == True)
        active_services = [s for s in services if s.get('active') is True]
        print(f"  Active services: {len(active_services)}")
        
        # Show all services with their status
        print("\nAll services:")
        for service in services:
            status = "ACTIVE" if service.get('active') is True else "ARCHIVED"
            print(f"  - {service.get('name')} (ID: {service['id']}) - {status}")
        
        # Display which service will be used
        if active_services:
            print(f"\n✓ First active service that will be used in tests:")
            print(f"  {active_services[0].get('name')} (ID: {active_services[0]['id']})")
        else:
            print("\n⚠️ No active services found!")
    else:
        print("No services found")
except Exception as e:
    print(f"❌ Error: {e}")

## Test 2: Get User Reports

This is the main test for the debug tool. We'll test fetching user reports for a specific year.

In [ ]:
# Test for current year
print(f"Testing user reports for year: {current_year}")

try:
    reports_response = client.get_user_reports(year=current_year)
    print("✓ Successfully retrieved user reports")
    print(f"\nRaw response:")
    print(json.dumps(reports_response, indent=2))
    
    # Show response structure
    print(f"\n📊 Response keys: {list(reports_response.keys())}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 3: Get User Reports with Different Type Levels

The Clockodo API supports different type levels for aggregation.

In [ ]:
# Test different type levels (0, 1, 2, 3)
for type_level in [0, 1, 2, 3]:
    print(f"\n{'='*60}")
    print(f"Testing type_level={type_level}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=current_year, type_level=type_level)
        print(f"✓ Success")
        print(f"Response keys: {list(reports.keys())}")
        
        # Show a sample of the data
        print(f"\nSample data:")
        print(json.dumps(reports, indent=2)[:1000] + "..." if len(json.dumps(reports)) > 1000 else json.dumps(reports, indent=2))
        
    except Exception as e:
        print(f"❌ Error: {e}")

## Test 4: Get User Reports for Specific User

If you have multiple users, test fetching reports for a specific user.

In [ ]:
# First get the user ID from the users list
try:
    users_response = client.list_users()
    if 'users' in users_response and len(users_response['users']) > 0:
        user_id = users_response['users'][0]['id']
        print(f"Testing reports for user_id: {user_id}")
        
        reports = client.get_user_reports(year=current_year, user_id=user_id)
        print("✓ Successfully retrieved user-specific reports")
        print(f"\nResponse:")
        print(json.dumps(reports, indent=2))
    else:
        print("No users found to test with")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 5: Test Debug Tool Function

Test the actual debug tool function from the module.

In [ ]:
from clockodo_mcp.tools.debug_tools import get_raw_user_reports

print(f"Testing debug tool for year {current_year}")

try:
    result = get_raw_user_reports(year=current_year)
    print("✓ Debug tool executed successfully")
    print(f"\nResult:")
    print(json.dumps(result, indent=2))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 6: Inspect HTTP Headers

Check what headers are being sent with requests.

In [ ]:
print("Headers being sent with requests:")
headers = client.default_headers
print(json.dumps(headers, indent=2))

## Test 7: Test Different Years

Test fetching reports for different years to see what data is available.

In [ ]:
# Test last 3 years
test_years = [current_year - 2, current_year - 1, current_year]

for year in test_years:
    print(f"\n{'='*60}")
    print(f"Year: {year}")
    print(f"{'='*60}")
    
    try:
        reports = client.get_user_reports(year=year)
        print(f"✓ Success")
        
        # Try to extract some summary info
        if 'userreports' in reports:
            print(f"  Found 'userreports' key")
            print(f"  Data type: {type(reports['userreports'])}")
            if isinstance(reports['userreports'], list):
                print(f"  Number of entries: {len(reports['userreports'])}")
        else:
            print(f"  Response keys: {list(reports.keys())}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

## Test 8: User Service Identification

Test the UserService and how it identifies the current user ID.

In [ ]:
try:
    user_id = user_service.get_current_user_id()
    print(f"✓ Current user ID: {user_id}")
    print(f"  Identified by email: {client.api_user}")
except Exception as e:
    print(f"❌ Error: {e}")

## Test 9: Clock Operations

Test getting, starting, and stopping the clock.

In [ ]:
# Initialize clock_entry_id for cleanup
clock_entry_id = None

try:
    # Get active service ID dynamically
    first_service_id = get_first_active_service_id()
    
    # 1. Get current clock
    clock_resp = user_service.get_my_clock()
    print("✓ Successfully retrieved current clock status")
    print(json.dumps(clock_resp, indent=2))
    
    # 2. Check if clock is running
    if clock_resp.get('running'):
        print("\n⚠️ Clock is currently running. Stopping it first...")
        user_service.stop_my_clock()
        print("✓ Clock stopped")
    else:
        print("\n✓ Clock is NOT running.")
        
    # Note: Starting/Stopping the clock will create real data in Clockodo!
    print(f"\nTesting clock start (WILL CREATE REAL DATA)...")
    print(f"  Customer ID: {first_customer_id}")
    print(f"  Service ID: {first_service_id}")
    
    if first_customer_id and first_service_id:
        result = user_service.start_my_clock(
            customers_id=first_customer_id, 
            services_id=first_service_id
        )
        print("✓ Successfully started clock")
        print(json.dumps(result, indent=2))
        
        # Capture the clock entry ID for cleanup
        if result.get('running') and result['running'].get('id'):
            clock_entry_id = result['running']['id']
            print(f"\n✓ Captured clock entry ID: {clock_entry_id}")
        
        # Stop it immediately
        print("\n  Stopping clock immediately...")
        stop_result = user_service.stop_my_clock()
        print("✓ Clock stopped")
        
        # The stopped entry ID is what we need to delete
        if stop_result.get('stopped') and stop_result['stopped'].get('id'):
            clock_entry_id = stop_result['stopped']['id']
            print(f"✓ Clock entry ID to cleanup: {clock_entry_id}")
    else:
        print("⚠️ Skipping clock start: No customer or active service available")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 10: Absences (Vacation)

Test listing absences and adding a vacation.

In [ ]:
# Initialize last_absence_id at the start
last_absence_id = None

try:
    # List absences for current year
    absences = client.list_absences(year=current_year)
    print(f"✓ Successfully retrieved absences for {current_year}")
    print(f"  Found {len(absences.get('absences', []))} absence(s)")
    
    # Show first absence if any
    if absences.get('absences'):
        print("\nSample absence:")
        print(json.dumps(absences['absences'][0], indent=2))
        
    # Note: add_my_vacation will create a real absence!
    # The created absence will be auto-approved (status: 1)
    # To delete it, you need to decline it first or delete it via the UI
    print("\nTesting add vacation (WILL CREATE REAL DATA)...")
    absence_result = user_service.add_my_vacation(date_since="2025-12-24", date_until="2025-12-26")
    print(json.dumps(absence_result, indent=2))
    if "absence" in absence_result:
        last_absence_id = absence_result["absence"]["id"]
        print(f"\nCaptured absence ID for cleanup: {last_absence_id}")
    
except Exception as e:
    print(f"❌ Error: {e}")

## Test 11: Time Entries

Test listing and adding time entries.

In [ ]:
# Initialize last_entry_id at the start
last_entry_id = None

try:
    # Get active service ID dynamically
    first_service_id = get_first_active_service_id()
    
    # List entries for today - USE ISO 8601 FORMAT: YYYY-MM-DDTHH:MM:SSZ
    today = datetime.now().strftime('%Y-%m-%d')
    time_since = f"{today}T00:00:00Z"
    time_until = f"{today}T23:59:59Z"
    
    entries = user_service.get_my_entries(time_since=time_since, time_until=time_until)
    print(f"✓ Successfully retrieved entries for {today}")
    print(f"  Found {len(entries.get('entries', []))} entry(s)")
    
    if entries.get('entries'):
        print("\nSample entry:")
        print(json.dumps(entries['entries'][0], indent=2))
        
    # Note: add_my_entry will create a real time entry!
    print("\nTesting add entry (WILL CREATE REAL DATA)...")
    print(f"  Customer ID: {first_customer_id}")
    print(f"  Service ID: {first_service_id}")
    print(f"  Time range: {today}T09:00:00Z to {today}T10:00:00Z")
    
    if first_customer_id and first_service_id:
        entry_result = user_service.add_my_entry(
            customers_id=first_customer_id, 
            services_id=first_service_id,
            billable=1,
            time_since=f"{today}T09:00:00Z",
            time_until=f"{today}T10:00:00Z",
            text="Manual test entry from notebook"
        )
        print("✓ Successfully added entry")
        print(json.dumps(entry_result, indent=2))
        
        if "entry" in entry_result:
            last_entry_id = entry_result["entry"]["id"]
            print(f"\n✓ Captured entry ID for cleanup: {last_entry_id}")
    else:
        print("⚠️ Skipping add entry: No customer or active service available")

except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 13: Team Leader - List Pending Vacation Requests

As a team leader, you can see all pending vacation requests.


In [ ]:
try:
    # List all pending vacation requests (status = 0)
    pending_requests = team_leader_service.list_pending_vacations(current_year)
    print(f"✓ Found {len(pending_requests)} pending vacation request(s)")
    
    if pending_requests:
        print("\nPending requests:")
        for req in pending_requests:
            print(json.dumps(req, indent=2))
    else:
        print("\nNo pending vacation requests at this time")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Test 14: Team Leader - Create Team Member Vacation

Create a vacation for a team member (requires knowing user ID).


In [ ]:
# Initialize variable for cleanup
team_vacation_id = None

try:
    # 1. Use TEST_TEAM_MEMBER_ID if provided, otherwise discover automatically
    if team_member_id:
        print(f"✓ Using team member ID from environment: {team_member_id}")
    else:
        print("🔍 Discovering a non-owner employee for testing...")
        users_resp = client.list_users()
        if 'users' in users_resp:
            # Find a user who is not the current user (owner)
            my_id = user_service.get_current_user_id()
            other_users = [u for u in users_resp['users'] if u['id'] != my_id]
            
            if other_users:
                team_member_id = other_users[0]['id']
                team_member_name = other_users[0]['name']
                print(f"✓ Found team member: {team_member_name} (ID: {team_member_id})")
            else:
                print("⚠️ No other users found. Falling back to current user.")
                team_member_id = my_id
    
    if team_member_id:
        print(f"Creating vacations for user ID: {team_member_id}")
        
        # 1. Create a vacation for testing approval
        vacation_result = team_leader_service.create_team_vacation(
            user_id=team_member_id,
            date_since="2026-01-10",
            date_until="2026-01-15",
            absence_type=1,  # Vacation
            auto_approve=False  # Keep pending to test approval
        )
        
        if "absence" in vacation_result:
            team_vacation_id = vacation_result["absence"]["id"]
            print(f"✓ Created vacation for approval test (ID: {team_vacation_id})")

        # 2. Create a vacation for testing rejection
        vacation_reject_result = team_leader_service.create_team_vacation(
            user_id=team_member_id,
            date_since="2026-02-10",
            date_until="2026-02-15",
            absence_type=1,  # Vacation
            auto_approve=False  # Keep pending to test rejection
        )
        
        if "absence" in vacation_reject_result:
            team_vacation_id_reject = vacation_reject_result["absence"]["id"]
            print(f"✓ Created vacation for rejection test (ID: {team_vacation_id_reject})")
    else:
        print("❌ Could not determine user ID for test")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Test 15: Team Leader - Approve/Reject Vacation

Demonstrate approving or rejecting a vacation request.
NOTE: This requires an actual pending vacation (status=0).


In [ ]:
try:
    # Use the vacation ID from Test 14 automatically
    absence_id_to_approve = team_vacation_id
    
    if absence_id_to_approve:
        print(f"Approving vacation ID: {absence_id_to_approve}")
        result = team_leader_service.approve_vacation(absence_id_to_approve)
        print("✓ Vacation approved")
        print(json.dumps(result, indent=2))
    else:
        print("ℹ️  No absence ID available for approval test")
    
    # Use the second vacation ID from Test 14 automatically
    absence_id_to_reject = team_vacation_id_reject
    
    if absence_id_to_reject:
        print(f"\nRejecting vacation ID: {absence_id_to_reject}")
        result = team_leader_service.reject_vacation(absence_id_to_reject)
        print("✓ Vacation rejected")
        print(json.dumps(result, indent=2))
    else:
        print("ℹ️  No absence ID available for rejection test")
    
except Exception as e:
    print(f"❌ Error: {e}")


## Test 16: Team Leader - Edit Team Member Time Entry

Edit a time entry for a team member.


In [ ]:
try:
    # Use the entry ID from Test 11 if available
    if last_entry_id:
        print(f"Editing time entry {last_entry_id}...")
        
        # Edit the entry description
        edit_result = team_leader_service.edit_team_entry(
            last_entry_id,
            {"text": "Updated by team leader"}
        )
        
        print("✓ Successfully edited team member entry")
        print(json.dumps(edit_result, indent=2))
    else:
        print("No entry ID available from previous tests")
        print("Create an entry first in Test 11, then run this cell")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()


## Test 17: Cleanup

Clean up all time entries and vacations created during testing.


In [ ]:
try:
    # 1. Cleanup Own Clock Entry (from Test 9)
    if 'clock_entry_id' in globals() and clock_entry_id:
        print(f"Cleaning up clock entry {clock_entry_id}...")
        user_service.delete_my_entry(clock_entry_id)
        print("✓ Successfully deleted clock entry")
    
    # 2. Cleanup Own Time Entry (from Test 11/16)
    if last_entry_id:
        print(f"Cleaning up time entry {last_entry_id}...")
        user_service.delete_my_entry(last_entry_id)
        print("✓ Successfully deleted time entry")
    
    # 3. Cleanup Own Absence (from Test 10)
    if last_absence_id:
        print(f"Cleaning up absence {last_absence_id}...")
        try:
            user_service.cancel_my_vacation(last_absence_id)
            user_service.delete_my_vacation(last_absence_id)
            print("✓ Successfully deleted own absence")
        except Exception:
            print(f"⚠️ Could not delete absence {last_absence_id} (requires admin rights)")
    
    # 4. Cleanup Team Vacations (from Test 14)
    if team_vacation_id:
        print(f"Cleaning up approved team vacation {team_vacation_id}...")
        client.edit_absence(team_vacation_id, {"status": 3})
        client.delete_absence(team_vacation_id)
        print("✓ Successfully deleted approved team vacation")
    
    if team_vacation_id_reject:
        print(f"Cleaning up rejected team vacation {team_vacation_id_reject}...")
        client.delete_absence(team_vacation_id_reject)
        print("✓ Successfully deleted rejected team vacation")
        
except Exception as e:
    print(f"❌ Error during cleanup: {e}")


## Test 18: HR Analytics - Overtime Compliance

Check which employees have excessive overtime.


In [ ]:
try:
    print(f"Checking overtime compliance for {current_year} (threshold: 80h)...")
    overtime_results = hr_service.check_overtime_compliance(current_year, max_overtime_hours=80)
    
    print(f"✓ Found {overtime_results['total_violations']} violation(s)")
    if overtime_results['violations']:
        print("\nViolations:")
        for v in overtime_results['violations']:
            print(f"  - {v['user_name']}: {v['overtime_hours']}h overtime (Limit: {v['threshold']}h)")
            
except Exception as e:
    print(f"❌ Error: {e}")


## Test 19: HR Analytics - Vacation Compliance

Check which employees have vacation compliance issues.


In [ ]:
try:
    print(f"Checking vacation compliance for {current_year}...")
    # You can customize these thresholds
    MIN_VACATION = 5
    MAX_REMAINING = 2
    
    print(f"Settings: Min used: {MIN_VACATION}, Max remaining: {MAX_REMAINING}")
    
    vacation_results = hr_service.check_vacation_compliance(
        current_year, 
        min_vacation_days=MIN_VACATION, 
        max_vacation_remaining=MAX_REMAINING
    )
    
    print(f"✓ Found {vacation_results['total_violations']} violation(s)")
    if vacation_results['violations']:
        print("\nViolations:")
        for v in vacation_results['violations']:
            print(f"  - {v['user_name']}: {v['violation_type']}")
            print(f"    Used: {v['used_days']} days, Remaining: {v['remaining_days']} days")
            
except Exception as e:
    print(f"❌ Error: {e}")


## Test 20: HR Analytics - Summary Report

Get a complete HR compliance summary.


In [ ]:
try:
    print(f"Generating complete HR summary for {current_year}...")
    summary = hr_service.get_hr_summary(current_year)
    
    print(f"✓ Summary generated")
    print(f"  Total Employees: {summary['total_employees']}")
    print(f"  Employees with violations: {summary['total_employees_with_violations']}")
    
    if summary['employees_with_violations']:
        print("\nDetailed Violations:")
        print(json.dumps(summary['employees_with_violations'], indent=2))
        
except Exception as e:
    print(f"❌ Error: {e}")
